# Computer Networks Computer Assignment 2
## Arman Rostami, Parsa Ghorbani

### Initialization

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
SIMULATION_COUNT = 10
TCL_FILENAME = "congestion.tcl"
OUTPUT_ENDING = ".csv"
CWND_DATA_NAME = "cwnd"
GOODPUT_DATA_NAME = "goodput"
RTT_DATA_NAME = "rtt"

In [3]:
plt.rcParams["figure.figsize"] = (14, 5) 

### Simulator

In [4]:
class Simulator:
    def __init__(self, congestion_algorithm):
        self.congestion_algorithm = congestion_algorithm
        self.data = {}

In [5]:
def train(self):
    for i in range(0, SIMULATION_COUNT):
        os.system("ns " + TCL_FILENAME + " " + self.congestion_algorithm)
        self.update_data(i, CWND_DATA_NAME)
        self.update_data(i, GOODPUT_DATA_NAME)
        self.update_data(i, RTT_DATA_NAME)
        
Simulator.train = train

In [6]:
def plot_data(self, data_1, data_2):
    plt.plot(data_1, "b", label="Top flow")
    plt.plot(data_2, "r", label="Bottom flow")
    plt.xlabel("time")
    plt.ylabel(data_1.name)
    plt.legend()

Simulator.plot_data = plot_data

In [7]:
def get_dict_key(self, data_name, index):
    return data_name + "_" + str(index)

Simulator.get_dict_key = get_dict_key

In [8]:
def update_data(self, simulation_index, data_name):
    new_data_1 = pd.read_csv(data_name + "1" + OUTPUT_ENDING)
    new_data_2 = pd.read_csv(data_name + "2" + OUTPUT_ENDING)
        
    data_dict_key_1 = self.get_dict_key(data_name, 1)
    data_dict_key_2 = self.get_dict_key(data_name, 2)
    
    if simulation_index == 0:
        self.data[data_dict_key_1] = new_data_1[data_name]
        self.data[data_dict_key_2] = new_data_2[data_name]
    else:
        self.data[data_dict_key_1] += new_data_1[data_name]
        self.data[data_dict_key_2] += new_data_2[data_name]
        
    if simulation_index == SIMULATION_COUNT - 1:
        self.data[data_dict_key_1] /= SIMULATION_COUNT
        self.data[data_dict_key_2] /= SIMULATION_COUNT
    
Simulator.update_data = update_data

In [9]:
def plot_cwnd(self):
    cwnd_data_1 = self.data[self.get_dict_key(CWND_DATA_NAME, 1)]
    cwnd_data_2 = self.data[self.get_dict_key(CWND_DATA_NAME, 2)]
    self.plot_data(cwnd_data_1, cwnd_data_2)
    
Simulator.plot_cwnd = plot_cwnd

In [10]:
def plot_goodput(self):
    goodput_data_1 = self.data[self.get_dict_key(GOODPUT_DATA_NAME, 1)]
    goodput_data_2 = self.data[self.get_dict_key(GOODPUT_DATA_NAME, 2)]
    self.plot_data(goodput_data_1, goodput_data_2)
    
Simulator.plot_goodput = plot_goodput

In [11]:
def plot_rtt(self):
    rtt_data_1 = self.data[self.get_dict_key(RTT_DATA_NAME, 1)]
    rtt_data_2 = self.data[self.get_dict_key(RTT_DATA_NAME, 2)]
    self.plot_data(rtt_data_1, rtt_data_2)
    
Simulator.plot_rtt = plot_rtt

### Train Simulators

In [ ]:
new_reno_simulator = Simulator("Newreno")
new_reno_simulator.train()

In [ ]:
tahoe_simulator = Simulator("Tahoe")
tahoe_simulator.train()

In [ ]:
vegas_simulator = Simulator("Vegas")
vegas_simulator.train()

### CWND

#### Newreno

In [ ]:
new_reno_simulator.plot_cwnd()

#### Tahoe

In [ ]:
tahoe_simulator.plot_cwnd()

#### Vegas

In [ ]:
vegas_simulator.plot_cwnd()

### Goodput

#### Newreno

In [ ]:
new_reno_simulator.plot_goodput()

#### Tahoe

In [ ]:
tahoe_simulator.plot_goodput()

#### Vegas

In [ ]:
vegas_simulator.plot_goodput()

### RTT

#### Newreno

In [ ]:
new_reno_simulator.plot_rtt()

#### Tahoe

In [ ]:
tahoe_simulator.plot_rtt()

#### Vegas

In [ ]:
vegas_simulator.plot_rtt()